# Assignment 4


In this task you have to scrape all the results from "Extranjero". [Here is link](https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E). 
You have to **follow the setps as we did in Lecture 7** and get information from all tabs in "Continente", "País" and "Estado".   

1. Take the data table TOTAL DE VOTOS. 
2. Store the four columns ORGANIZACIONES POLÍTICAS, TOTAL, %VÁLIDOS and %EMITIDOS. Rename these columns to political_parties, total, share_valid, share_emitied, respectively. 
3. Add the columns continent, country, state, ubigeo, and link. These last two items you can extract from the url. 
4. Append all the tables in just one DataFrame and export an excel file and save it in the folder [\_data\_results](https://github.com/alexanderquispe/Diplomado_PUCP/tree/main/Lecture_7/_data_results). 
5. The name of your Excel file should be the same as the name of your branches. 

In [90]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
import pandas as pd
from selenium.webdriver.support.ui import Select

In [31]:
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()

url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E'
driver.get( url )



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Work\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\Work\AppData\Local\Temp/ipykernel_10172/3890519660.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome( ChromeDriverManager().install() )


In [95]:
col_names = 'political_parties, total, share_valid, share_emitied'.split( ', ' )
all_tables = {}
cont = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select' ) )
num_cont_opt = len( cont.options )

for icont in range( num_cont_opt ):
    cont = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select' ) )
    continent = cont.options[ icont ]
    cont_name = continent.text
    
    if cont_name != '--TODOS--':
        continent.click()
        coun = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select' ) )
        num_coun_opt = len( coun.options )
        
        for icoun in range( num_coun_opt ):
            coun = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select' ) )
            country = coun.options[ icoun ]
            coun_name = country.text
            
            if coun_name != '--TODOS--':                
                country.click()
                stat = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select' ) )
                num_stat_opt = len( stat.options )
                
                for istat in range( num_stat_opt ):
                    stat = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select' ) )
                    state = stat.options[ istat ]
                    stat_name = state.text
                    
                    if stat_name != '--TODOS--':
                        state.click()

                        link = driver.current_url
                        ubigeo = link.split("/")[ -1 ]

                        table_path = driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div' )
                        table_html = table_path.get_attribute( 'innerHTML' )
                        table = pd.read_html( table_html )[ 0 ].iloc[ 1:, 2: ]
                        table.columns = col_names
                        table[ 'continent' ] = cont_name
                        table[ 'country' ] = coun_name
                        table[ 'state' ] = stat_name
                        table[ 'ubigeo' ] = ubigeo
                        table[ 'link' ] = link

                        all_tables[ ubigeo ] = table

C:\Users\Work\AppData\Local\Temp/ipykernel_10172/2114205303.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cont = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select' ) )
C:\Users\Work\AppData\Local\Temp/ipykernel_10172/2114205303.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cont = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select' ) )
C:\Users\Work\AppData\Local\Temp/ipykernel_10172/2114205303.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  coun = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select' ) )
C:\Users\Work\A

In [96]:
results = pd.concat( all_tables.values() ).reset_index( drop = True )

In [98]:
results.to_excel( r'../_data_results/group_3_ass_4_rodrigo.xlsx' , index = False )

In [99]:
driver.close()